In [1]:
from API_call import *
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd
from database import *

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [4]:
Base.metadata.create_all(db.db_engine)
Base.metadata.bind=db.db_engine

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [6]:
def insert(call,session):
    mappers = [
        Game,Play,
        Pitch,
        Action,Runner,Credit,
        Matchup,Venue,
        Team,GameTeamLink,TeamRecord,
        Player
    ]
    for mapper in mappers:
        table = mapper.__tablename__
        records = call.__dict__[table]
        if mapper == Runner:
            for record in records:
                if record['movement_start']==None:
                    record['movement_start']='Home'
        session.bulk_insert_mappings(mapper,records)
    for mapper in [PitchData,HitData]:
        cols = [x.name for x in mapper.__table__.c]
        records = call.__dict__['pitches']
        df = pd.DataFrame.from_records(records)
        df = df[cols]
        df.columns = [col.replace('pitchData_','') for col in df.columns]
        df.columns = [col.replace('hitData_','') for col in df.columns]
        df.dropna(axis=0,thresh=5,inplace=True)
        session.bulk_insert_mappings(mapper,df.to_dict(orient='records'))
    session.commit()
        
        

In [7]:
test_games = gamePks['2019']

In [8]:
# for call in calls:
#     try:
#         insert(call,session)
#     except:
#         session.rollback()
#         continue

In [9]:
session.query(Game).all()

[<Game(pk='567378',id='2019/05/04/atlmlb-miamlb-1')>,
 <Game(pk='566194',id='2019/05/04/oakmlb-pitmlb-1')>,
 <Game(pk='564944',id='2019/05/04/tbamlb-balmlb-1')>,
 <Game(pk='567078',id='2019/05/04/wasmlb-phimlb-1')>,
 <Game(pk='564752',id='2019/05/04/houmlb-anamlb-1')>,
 <Game(pk='565235',id='2019/05/04/sfnmlb-cinmlb-1')>,
 <Game(pk='567280',id='2019/05/04/bosmlb-chamlb-1')>,
 <Game(pk='567571',id='2019/05/04/nynmlb-milmlb-1')>,
 <Game(pk='566781',id='2019/05/04/tormlb-texmlb-1')>,
 <Game(pk='565427',id='2019/05/04/arimlb-colmlb-1')>,
 <Game(pk='566291',id='2019/05/04/lanmlb-sdnmlb-1')>,
 <Game(pk='564945',id='2019/07/13/tbamlb-balmlb-1')>,
 <Game(pk='567379',id='2019/05/05/atlmlb-miamlb-1')>,
 <Game(pk='565329',id='2019/05/05/seamlb-clemlb-1')>,
 <Game(pk='565524',id='2019/05/05/kcamlb-detmlb-1')>,
 <Game(pk='566195',id='2019/05/05/oakmlb-pitmlb-1')>,
 <Game(pk='567079',id='2019/05/05/wasmlb-phimlb-1')>,
 <Game(pk='567281',id='2019/05/05/bosmlb-chamlb-1')>,
 <Game(pk='567572',id='2019/

In [10]:
def insert_game(gamePk):
    q = db.db_engine.execute('select pk from games').fetchall()
    existing_pks = [x[0] for x in q]
    
    if gamePk in existing_pks:
        return f"{gamePk} already exists"
    else:
        call = API_call(gamePk)

        mappers = [
            Game,Play,
            Pitch,
            Action,Runner,Credit,
            Matchup,Venue,
            Team,GameTeamLink,TeamRecord,
            Player,PitchData,HitData
        ]

        for mapper in mappers:
            insert_values = []

            table = mapper.__table__
            cols = [x.name for x in table.c]

            if mapper == HitData or mapper == PitchData:
                records = call.__dict__['pitches']
            else:
                records = call.__dict__[table.name]
            for record in records:
                insert_value = {}
                for k in cols:
                    try:
                        insert_value[k]=record[k]
                    except KeyError:
                        insert_value[k]=None
                insert_values.append(insert_value)

            conn = db.db_engine.connect()    
            conn.execute(table.insert(),insert_values)


In [11]:
#[insert_game(game) for game in test_games]

In [19]:
#TEST: which columns, if any, don't have any values
dfs = {}
for table in db.db_engine.table_names():
    df = pd.read_sql(f"select * from {table}",db.db_engine)
    dfs[table]=df


for k in dfs.keys():
    df = dfs[k]
    for col in df.columns:
        if df[col].isna().sum()==df.shape[0]:
            print(k,col)

In [15]:
pks = [x[0] for x in session.query(Game.pk).all()]

calls = [API_call(pk) for pk in pks]

dfs_fromApi = {}
for k in calls[0].__dict__.keys():
    if k[0] != '_':
        records = [x for call in calls for x in call.__dict__[k]]
        df = pd.DataFrame.from_records(records)
        dfs_fromApi[k] = df

In [21]:
dfs['pitch_data']

,gamePk,atBatIndex,playEndTime,index,pitchData_startSpeed,pitchData_endSpeed,pitchData_strikeZoneTop,pitchData_zone,pitchData_typeConfidence,pitchData_plateTime,...,pitchData_coordinates_vZ0,pitchData_coordinates_x,pitchData_coordinates_y,pitchData_coordinates_x0,pitchData_coordinates_y0,pitchData_coordinates_z0,pitchData_coordinates_aX,pitchData_breaks_breakY,pitchData_breaks_spinRate,pitchData_breaks_spinDirection
0,567378,0,2019-05-04T22:11:11.000Z,3,91.3,83.0,3.20,11.0,2.0,0.41,...,-3.24,163.19,130.76,-1.39,50.0,6.14,-11.03,24.0,2168.0,208.0
1,567378,0,2019-05-04T22:11:11.000Z,4,91.5,83.2,3.29,11.0,2.0,0.41,...,-4.77,122.51,149.59,-1.42,50.0,6.04,-8.30,24.0,2205.0,200.0
2,567378,0,2019-05-04T22:11:11.000Z,5,80.4,74.1,3.22,13.0,2.0,0.47,...,-4.63,124.25,225.90,-1.54,50.0,5.92,0.43,24.0,2331.0,12.0
3,567378,1,2019-05-04T22:12:33.000Z,0,91.5,83.3,3.34,6.0,2.0,0.41,...,-6.98,106.85,172.51,-1.21,50.0,6.01,-10.27,24.0,2139.0,205.0
4,567378,1,2019-05-04T22:12:33.000Z,1,84.7,77.3,3.27,12.0,2.0,0.45,...,0.40,116.49,133.36,-1.39,50.0,6.20,4.74,24.0,2135.0,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15470,565429,83,2019-05-08T04:04:02.000Z,7,95.3,88.7,3.35,12.0,NaN,0.40,...,-0.79,82.26,134.02,2.31,50.0,5.66,6.74,24.0,2181.0,148.0
15471,565429,84,2019-05-08T04:05:21.000Z,0,93.1,86.7,3.35,3.0,NaN,0.41,...,-3.42,98.10,159.20,2.27,50.0,5.66,6.94,24.0,2140.0,151.0
15472,565429,84,2019-05-08T04:05:21.000Z,1,93.7,87.2,3.36,5.0,NaN,0.40,...,-4.50,114.46,165.71,2.10,50.0,5.74,5.99,24.0,2152.0,157.0
15473,565429,85,2019-05-08T04:06:18.000Z,0,87.3,81.6,3.31,11.0,NaN,0.43,...,-2.46,150.57,168.62,2.40,50.0,5.47,-1.30,24.0,2245.0,198.0


In [23]:
[insert_game(game) for game in gamePks['2019']]

KeyError: 'playEndTime'